In [1]:
# 自动计算cell的计算时间
%load_ext autotime

%matplotlib inline
%config InlineBackend.figure_format='svg' #矢量图设置，让绘图更清晰

time: 1.44 s (started: 2021-08-18 23:17:04 +08:00)


In [ ]:
%%bash

# 增加更新
git add *.ipynb *.md

git remote -v

git commit -m '更新 ch15 #1  Aug 18, 2021'

git push origin master